# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df = data.groupby(['CustomerID', 'ProductName'])['Quantity'].sum()
df.head()

CustomerID  ProductName                  
33          Apricots - Dried                 1
            Assorted Desserts                1
            Bandage - Flexible Neon          1
            Bar Mix - Pina Colada, 355 Ml    1
            Beans - Kidney, Canned           1
Name: Quantity, dtype: int64

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
table = pd.pivot_table(data, values='Quantity', index=['ProductName'], columns=['CustomerID'], aggfunc=np.sum, fill_value=0)
table.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
dist = squareform(pdist(table.T,'euclidean'))
dist

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
dist = pd.DataFrame(1/(1 + squareform(pdist(table.T,'euclidean'))), index=table.columns, columns=table.columns)
dist.head(6)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
464,0.082709,0.075255,0.080350,0.078187,0.078711,1.000000,0.074798,0.084626,0.080070,0.078187,...,0.004816,0.004687,0.004416,0.005038,0.004308,0.004533,0.004578,0.004363,0.004167,0.004347


In [8]:
similar = dist[33].sort_values(ascending=False)[1:].head()
similar

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
records = df.loc[similar.index]
records = pd.DataFrame(records)
records.head(10)

Quantity
CustomerID ProductName                            
264        Apricots - Halves                     1
           Apricots Fresh                        1
           Bacardi Breezer - Tropical            1
           Bagel - Plain                         1
           Banana - Leaves                       1
           Bananas                               1
           Bandage - Flexible Neon               1
           Beef - Tenderlion, Center Cut         1
           Berry Brulee                          1
           Bread - Roll, Canadian Dinner         1

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
records_agg = records.groupby('ProductName').agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False)
records_agg.head()

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [11]:
records_agg2 = table[table[33]==0]
recommend = records_agg2.merge(records_agg, left_on='ProductName', right_on='ProductName').sort_values('Quantity', ascending=False)
recommend = recommend[33].head()
recommend

ProductName
Wine - Blue Nun Qualitatswein    0
Soup - Campbells Bean Medley     0
Wine - Ej Gallo Sierra Valley    0
Butter - Unsalted                0
Yoghurt Tubes                    0
Name: 33, dtype: int64

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [12]:
recommendations = {}
custom_id = set(list(data.CustomerID))

In [13]:
for i in custom_id:
    similar = dist[i].sort_values(ascending=False)[0:5]
    records = df.loc[similar.index]
    recordsbought = records.groupby(["ProductName"]).sum(level='Quantity').sort_values(ascending=False)
    rec = recordsbought[~recordsbought.index.isin(df.loc[i].index)].head()
    recommendations[i] = rec.to_dict()

for e in recommendations:
    recommendations[e] = [x for x in recommendations[e]]

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [14]:
dfrecommendations = pd.DataFrame(recommendations).T
dfrecommendations = dfrecommendations.rename(columns = {0: "Product #1", 1: "Product #2", 2: "Product #3", 3: "Product #4", 4: "Product #5"})
dfrecommendations.head()

,Product #1,Product #2,Product #3,Product #4,Product #5
83973,Cocoa Butter,Veal - Slab Bacon,Ezy Change Mophandle,Bandage - Fexible 1x3,Puree - Passion Fruit
59399,"Shrimp - Baby, Warm Water",Nantucket - Pomegranate Pear,Bagel - Plain,Snapple - Iced Tea Peach,Towels - Paper / Kraft
92168,Cup - Translucent 7 Oz Clear,Snapple Lemon Tea,"Veal - Brisket, Provimi,bnls",Pop Shoppe Cream Soda,Bread - Calabrese Baguette
49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Cheese - Parmesan Grated,Puree - Mocha,"Chicken - Leg, Boneless"
18441,Soupfoamcont12oz 112con,Sausage - Breakfast,Flour - Pastry,Soup Knorr Chili With Beans,Foam Cup 6 Oz


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [15]:
jaccard = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'jaccard'))),index=table.columns, columns=table.columns)
recommendations2 = {}
custom_id2 = set(list(data.CustomerID))

In [16]:
for i in custom_id2:
    similar = jaccard[i].sort_values(ascending=False)[0:5]
    records = df.loc[similar.index]
    recordsbought = records.groupby(["ProductName"]).sum(level='Quantity').sort_values(ascending=False)
    rec = recordsbought[~recordsbought.index.isin(df.loc[i].index)].head()
    recommendations2[i] = rec.to_dict()

for e in recommendations2:
    recommendations2[e] = [x for x in recommendations2[e]]

In [17]:
dfrecommendations2 = pd.DataFrame(recommendations2).T
dfrecommendations2 = dfrecommendations2.rename(columns = {0: "Product #1", 1: "Product #2", 2: "Product #3", 3: "Product #4", 4: "Product #5"})
dfrecommendations2.head()

,Product #1,Product #2,Product #3,Product #4,Product #5
83973,"Chocolate - Semi Sweet, Calets",Milk - 1%,Pork - Belly Fresh,Towels - Paper / Kraft,Bread - Rye
59399,"Cheese - Brie, Triple Creme",Kellogs All Bran Bars,"Rum - Coconut, Malibu",Baking Powder,"Oil - Shortening,liqud, Fry"
92168,Blackberries,Wiberg Super Cure,Coffee - Irish Cream,Olives - Stuffed,Tea - English Breakfast
49159,Flavouring - Orange,Macaroons - Two Bite Choc,Mussels - Cultivated,Cookie Dough - Double,Lettuce - Frisee
18441,Bread Foccacia Whole,Macaroons - Two Bite Choc,Broom - Corn,"Soup - Campbells, Lentil",Beer - Sleemans Cream Ale
